In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, simpledialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import threading
import time
import pandas as pd

# test 2
forecasted_prices_path = 'forecasted_prices.csv'
forecasted_data = pd.read_csv(forecasted_prices_path)
forecasted_prices_list = forecasted_data['0'].tolist()

class Stock:
    def __init__(self, name, forecast_prices):
        self.name = name
        self.current_price = forecast_prices[0]  # Use the first forecasted price as initial
        self.price_history = [self.current_price]
        self.forecast_prices = forecast_prices[1:]  # Exclude the first value already used as initial

    def update_price_from_forecast(self):
        if self.forecast_prices:
            new_price = self.forecast_prices.pop(0) if self.forecast_prices else self.current_price
            self.current_price = new_price
            self.price_history.append(new_price)

def simulate_market(stocks):
    while True:
        for stock in stocks:
            stock.update_price_from_forecast()
        time.sleep(5)

class StockApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Stock Simulator")
        self.stocks = [
            Stock("Exch0", forecasted_prices_list),
        ]
        self.selected_stock = None
        self.balance = 1000
        self.portfolio = {stock.name: 0 for stock in self.stocks}
        self.create_widgets()
        self.update_balance_display()
        threading.Thread(target=simulate_market, args=(self.stocks,), daemon=True).start()
        self.update()

    def create_widgets(self):
        self.balance_label = tk.Label(self.master, text="")
        self.balance_label.pack()

        self.stock_listbox = tk.Listbox(self.master)
        self.stock_listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        for stock in self.stocks:
            self.stock_listbox.insert(tk.END, stock.name)
        self.stock_listbox.bind('<<ListboxSelect>>', self.on_stock_select)

        self.fig, self.ax = plt.subplots(figsize=(5, 3))
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.master)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)
        
        self.buy_button = tk.Button(self.master, text="Buy", command=self.buy_stock)
        self.buy_button.pack(side=tk.LEFT)

        self.sell_button = tk.Button(self.master, text="Sell", command=self.sell_stock)
        self.sell_button.pack(side=tk.RIGHT)

        # Current price label
        self.current_price_label = tk.Label(self.master, text="")
        self.current_price_label.pack()

    def on_stock_select(self, event=None):
        if event:
            selection = event.widget.curselection()
            if selection:
                index = selection[0]
                self.selected_stock = self.stocks[index]
        else:
            # Handle the case when called without an event
            index = self.stock_listbox.curselection()
            if index:
                self.selected_stock = self.stocks[index[0]]
    
        # Update for selected stock
        if self.selected_stock:
            self.ax.clear()
            self.ax.plot(self.selected_stock.price_history)
            self.ax.set_title(self.selected_stock.name)
            self.canvas.draw()
            # Update current price label
            self.current_price_label.config(text=f"Current Price: ${self.selected_stock.current_price}")

    def buy_stock(self):
        if self.selected_stock:
            num_stocks = simpledialog.askinteger("Buy Stocks", "Enter number of stocks to buy:")
            if num_stocks is not None and num_stocks > 0:
                total_cost = self.selected_stock.current_price * num_stocks
                if self.balance >= total_cost:
                    self.balance -= total_cost
                    self.portfolio[self.selected_stock.name] += num_stocks
                    self.update_balance_display()
                else:
                    messagebox.showwarning("Warning", "Not enough balance to buy!")
            elif num_stocks is not None:
                messagebox.showerror("Error", "Please enter a valid number of stocks to buy.")

    def sell_stock(self):
        if self.selected_stock:
            num_stocks = simpledialog.askinteger("Sell Stocks", "Enter number of stocks to sell:")
            if num_stocks is not None and num_stocks > 0:
                if self.portfolio[self.selected_stock.name] >= num_stocks:
                    total_revenue = self.selected_stock.current_price * num_stocks
                    self.balance += total_revenue
                    self.portfolio[self.selected_stock.name] -= num_stocks
                    self.update_balance_display()
                else:
                    messagebox.showwarning("Warning", "You do not own enough stocks to sell!")
            elif num_stocks is not None:
                messagebox.showerror("Error", "Please enter a valid number of stocks to sell.")

    def update_balance_display(self):
        self.balance_label.config(text=f"Balance: ${self.balance}")
        portfolio_text = "Portfolio: " + ", ".join(
            [f"{name}: {qty}" for name, qty in self.portfolio.items() if qty > 0])
        self.balance_label.config(text=f"Balance: ${self.balance}\n{portfolio_text}")

    def update(self):
        if self.selected_stock is not None:
            self.on_stock_select()

        self.master.after(5000, self.update)

if __name__ == "__main__":
    root = tk.Tk()
    app = StockApp(root)
    root.mainloop()
